<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [148]:
# импортируем необходимые библиотеки
import plotly.express as px
import plotly.graph_objects as go

import psycopg2
import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd

import requests
from bs4 import BeautifulSoup

In [149]:
# не забыть удалить)
#DBNAME = 
#USER = 
#PASSWORD = 
#HOST = 
#PORT = 

In [150]:
# подключаемся к базе данных
connected = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}'
engine = sqlalchemy.create_engine(connected)

# Юнит 3. Предварительный анализ данных

In [151]:
# Так как запросы однотипны, предварительно оптимизируем работу

# Напишем функцию принимающую SQL-запрос(в виде строки) и возвращающую результат запроса в DataFrame
def SqlRequest(sql):
  with engine.connect().execution_options(autocommit=True) as conn:
    return pd.read_sql(sql, con = conn)

# Напишем функцию возвращающую количество записей в таблице
def RecordCount(table):
  sql = f'''
    SELECT 
      count(*) as {table}_count 
    FROM public.{table}
  '''
  return SqlRequest(text(sql)).iloc[0,0]

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [152]:
RecordCount('vacancies')

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [153]:
RecordCount('employers')

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [154]:
RecordCount('areas')

1362

4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица industries).

In [155]:
RecordCount('industries')

294

#### Вывод по итогам предварительного анализа данных:
    
1. Рассматриваемый объем данных включает в себя около 50 000 вакансий, что составляет примерно 3,4% от всего количества предлагаемых вакансий в РФ по данным Минтруда.
2. Представлены вакансии из 1362 регионов, что сопоставимо с количеством городов в РФ (1118).
3. Вакансии на hh.ru размещают 23 500 работодателей.
4. Соискателям предлагаются вакансии в 294 сферах экономической деятельности (охвачено большинство существующих сфер экономической деятельности).  
Учитывая все вышесказанное можно сделать вывод о том, что рассматриваемые данные являются представительными с точки зрения распределения по регионам и сферам экономической деятельности. Однако необходимо исследовать неравномерность распределения вакансий по регионам, которая с большой вероятностью имеется.
Можно предположить, что количество вакансий и работодателей также является представительным, однако это требует дополнительного исследования.
****

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [156]:
sql_u4_1 = f'''
    SELECT 
      a.name as area,
      count(*) as cnt 
    FROM public.areas as a
    JOIN public.vacancies as v on a.id = v.area_id
    GROUP by a.name
    ORDER by cnt desc
    LIMIT 5
  '''

In [157]:
SqlRequest(text(sql_u4_1))

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [158]:
sql_u4_2 = f'''
    SELECT 
        count(*) as cnt
    FROM public.vacancies as v
    WHERE v.salary_from is not null 
        or v.salary_to is not null  
  '''

In [159]:
SqlRequest(text(sql_u4_2))

,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [160]:
sql_u4_3 = f'''
    SELECT 
        avg(salary_from) as avg_salary_from,
        avg(salary_to) as avg_salary_to
    FROM public.vacancies as v
  '''

In [161]:
SqlRequest(text(sql_u4_3)).round()

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [162]:
sql_u4_4 = f'''
    SELECT 
        v.schedule,
        v.employment,
        count(*) as vacancies_count
    FROM public.vacancies as v
    GROUP by v.schedule, v.employment
    ORDER by vacancies_count desc
  '''

In [163]:
SqlRequest(text(sql_u4_4))

,schedule,employment,vacancies_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [164]:
sql_u4_5 = f'''
    SELECT 
        v.experience,
        count(*) as vacancies_count
    FROM public.vacancies as v
    GROUP by v.experience
    ORDER by vacancies_count asc
  '''

In [165]:
SqlRequest(text(sql_u4_5))

,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


#### Вывод по итогам детального анализа данных:

1. Основная масса вакансий приходится на крупные города. Это можно объяснить следующими факторами: чем больше население, тем больше рабочих мест; управленческие структуры крупного бизнеса распологаются в крупных городах; сектор услуг лучше развит в крупных городах и т.д.
2. Наибольшее количество предлагаемых вакансий в Москве (5333), далее следуют Санкт-Петербург (2851), Минск (2112), Новосибирск(2006) и Алматы(1892). Оказывается вакансии предлагаются не только в регионах РФ, но и в соседних странах (Казахстан, Беларусь), что может быть связано с популярностью портала hh.ru в этих странах.
3. В примерно половине вакансий не указана зарплата. Это может быть связано с тем, что работодатель не хочет афишировать данные по предлагаемой зарплате или система оплаты труда в компании очень гибкая и решение о размере ЗП принимается после контакта с соискателем.
4. Средняя предлагаемая зарплата находится в диапазоне от 71065 до 110537 рублей для вакансий, в которых зарплата указана (24073 из 49197).
5. Самый распространенный график работы - Полная занятость и Полный рабочий день (35367). Самые эксклюзивные виды занятости связаны с проектной работой. Это связано с историческими традициями наемного труда в РФ. Новые формы трудовых отношений появляются, но пока они не распостранены.
6. Наименьшее количество вакансий требует опыт работы более 6 лет (1337), самые распотраненные требования к опыту от 1 года до 3 лет (26152), соискатели без опыта могут расчитывать только на 7197 вакансий. Это может быть связано с тем, что работодатели, размещая вакансии на hh.ru расчитывают привлечь в первую очередь молодых соискателей, которые непременно ищут работу с помощью подобных порталов. Работники с большим опытом видимо ищут работу другими способами, поэтому число вакансий для них минимально.
****

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [167]:
sql_u5_1 = f'''
    (SELECT 
        e.name,
        count(*) as vacancies_count
    FROM public.vacancies as v
    INNER JOIN public.employers as e on e.id = v.employer_id
    GROUP by e.name
    ORDER by vacancies_count desc
    LIMIT 1)
    
    UNION
    
    (SELECT 
        e.name,
        count(*) as vacancies_count
    FROM public.vacancies as v
    INNER JOIN public.employers as e on e.id = v.employer_id
    GROUP by e.name
    ORDER by vacancies_count desc
    LIMIT 1
    OFFSET 4)
    ORDER BY name DESC
  '''

In [168]:
SqlRequest(text(sql_u5_1))

,name,vacancies_count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [171]:
sql_u5_2 = f'''
    SELECT
        a.name, 
        count(distinct e.id) as employers_count,
        count(distinct v.id) as vacancies_count
    FROM public.areas as a
    LEFT JOIN public.employers as e on e.area = a.id
    LEFT JOIN public.vacancies as v on v.area_id = a.id
    WHERE v.id is null
    GROUP by a.name
    ORDER by employers_count desc
    LIMIT 1
  '''

In [172]:
SqlRequest(text(sql_u5_2))

,name,employers_count,vacancies_count
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [173]:
sql_u5_3 = f'''
    SELECT
        e.name, 
        count(distinct v.area_id) as areas_count
    FROM public.employers as e
    LEFT JOIN public.vacancies as v on v.employer_id = e.id
    GROUP by e.name
    ORDER by areas_count desc
  '''

In [174]:
SqlRequest(text(sql_u5_3))

,name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [175]:
sql_u5_4 = f'''
    SELECT
      count(e.id) as cnt
    FROM public.employers as e
    LEFT JOIN public.employers_industries as ei on ei.employer_id = e.id
    WHERE ei.employer_id is null
  '''

In [176]:
SqlRequest(text(sql_u5_4))

,cnt
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [184]:
sql_u5_5 = f'''
    SELECT
      e.name,
      count(distinct ei.industry_id) as cnt
    FROM public.employers as e
    LEFT JOIN public.employers_industries as ei on ei.employer_id = e.id
    WHERE ei.employer_id is not null
    GROUP by e.name
    HAVING count(distinct ei.industry_id) = 4
    ORDER by e.name asc
    LIMIT 1
    OFFSET 2
  '''

In [185]:
SqlRequest(text(sql_u5_5))

,name,cnt
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [64]:
sql_u5_6 = f'''
    SELECT
      count(e.name) as cnt
    FROM public.employers as e
    LEFT JOIN public.employers_industries as ei on ei.employer_id = e.id
    JOIN public.industries as i on i.id = ei.industry_id
    WHERE i.name = 'Разработка программного обеспечения'
  '''

In [186]:
SqlRequest(text(sql_u5_6))

,cnt
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [187]:
# Используем метод pandas read_html, который позволяет считывать все таблицы с указанного url в DataFrame

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
cities = list(pd.read_html(url)[1]['Город'])

In [188]:

# Запишем повторяющийся SQL-запрос в отдельную переменную и с помощью join получим список городов с вакансиями 
# из списка городов-миллионников, не забудем про кавычки в названиях.
com_sql = f'''
    FROM public.employers as e
    LEFT JOIN public.vacancies as v on v.employer_id = e.id
    JOIN public.areas as a on v.area_id = a.id 
    WHERE 
      e.name = 'Яндекс'
      and v.id is not null
      and a.name in ({','.join(f'{chr(0x27)}{c}{chr(0x27)}' for c in cities)})
    '''
# итоговый SQL-запрос
sql_u5_7 = f'''
    (SELECT
      a.name,
      count(v.id) as cnt
    {com_sql}
    GROUP BY 1
    ORDER BY 2 asc)
    
    UNION
    
    (SELECT
      'Total',
      count(v.id) as cnt
    {com_sql}
    )
    ORDER BY 2
  '''

In [189]:
SqlRequest(text(sql_u5_7))

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

#### Вывод по итогам анализа работодателей:

1. На портале hh.ru размещают вакансии крупные компании, средние бизнесы и индивидуальные предприниматели. Представлен весь спектр работодателей. Большее число вакансий на hh.ru размещено компанией Яндекс (1933). IT-компании охотно ищут сотрудников через портал hh.ru.
2. Наибольшее количество работодателей без активных вакансий из России (410). Это связано с тем, что работодатели не всегда указывают конкретный регион.
3. Компания Яндекс является чемпионом по количеству регионов с активными вакансиями (181). На втором месте - Ростелеком (152). Возможно это всязано с тем, что Яндекс расширяет бизнес в регионах. У Ростелекома причина иная, эта компания исторически широко представлена на всей территории РФ, возможно дело в текучке кадров.
4. У 8419 работодателей (36% от общего числа) не указана ни одна сфера деятельности. Это может быть связано с тем, что компания уверена в своей известности или в некомпетентности работников HR-отдела, размещающих вакансии на портале.
5. 1132 работодателя напротив указали 4 сферы деятельности, среди них компания 2ГИС.
6. 3553 работодателя помимо прочего занимаются разработкой ПО. Это 15% от общего числа представленых работодателей. Это можно объяснить тем, что IT-компании охотно ищут сотрудников через портал hh.ru.
7. Компания Яндекс размещает вакансии во всех городах-миллионниках РФ. Распределение вакансий достаточно равномерно (около 30) во всех городах кроме Екатеринбурга (39), Санкт-Петербурга (42) и Москвы (54). Скорее всего это связано с размерами офиса и количеством населения. Общее количество вакансий от Яндекса в городах-миллионниках РФ равно 485, что составляет 25% от всех вакансий Яндекса, представленных на hh.ru. Данный факт говорит о том, что компания Яндекс представлена не только в крупных городах, но и в других регионах РФ.
****

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [49]:
sql_u6_1 = f'''
    SELECT
      count(distinct v.id) as cnt
    FROM public.vacancies as v 
    WHERE v.name ~~* any (array[
      '%%data%%',
      '%%данн%%'
    ])
  '''

In [75]:
SqlRequest(text(sql_u6_1))

,cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [76]:
# условия для Junior пропишем в отдельной переменной
juniors ='''   
    (
    v.employment = 'Стажировка' 
    or v.experience = 'Нет опыта' 
    or v.name ~~* '%%junior%%'
    ) 
    '''
# условия для DS пропишем в отдельной переменной
DS_Only = '''    
    (
    v.name ~~* any (array[
      '%%data%%scientist%%',
      '%%data%%science%%',
      '%%исследователь%%данных%%',
      '%%machine%%learning%%',
      '%%машинн%%обучен%%'      
      ])
    or (
    v.name !~~* '%%HTML%%'
    and v.name ~~ '%%ML%%'
      )
    )
'''
# итоговый запрос
sql_u6_2 = f'''
    SELECT
      count(*) as cnt
    FROM public.vacancies as v
    WHERE
    {DS_Only}
    and
    {juniors}
  '''

In [77]:
SqlRequest(text(sql_u6_2))

,cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [78]:
# Из уже выбранных DS вакансий, выберем только те, в которых требуется наличие навыков SQL или postgreSQL
sql_u6_3 = f'''
    WITH DSOnly as (
    SELECT *
    FROM public.vacancies as v
    WHERE
    {DS_Only}
    )
    SELECT count(*) as cnt
    FROM DSOnly ds
    WHERE ds.key_skills ~~* any (array[
      '%%sql%%',
      '%%postgres%%'
    ])  
  '''

In [79]:
SqlRequest(text(sql_u6_3))

,cnt
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [80]:
sql_u6_4 = f'''
    WITH DSOnly as (
    SELECT *
    FROM public.vacancies as v
    WHERE
    {DS_Only}
    )
    SELECT count(*) as cnt
    FROM DSOnly ds
    WHERE ds.key_skills ~~* any (array[
      '%%python%%'
      ])   
  '''

In [81]:
SqlRequest(text(sql_u6_4))

,cnt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [82]:
# Из уже выбранных DS вакансий расчитываем среднее количество навыков

# Признак key_skills использует \t (CHR(9)) для разделения навыков, удаляем через replace \t 
# прибавляем 1, так как разделитель в конце не ставится и поэтому навыков на 1 больше, чем разделителей
# полученное число отнимаем от всей длины строки, тем самым получая в ответе количество указанных навыков

sql_u6_5 = f'''
    WITH DSOnly as (
    SELECT *
    FROM public.vacancies as v
    WHERE
    {DS_Only}
    )

    SELECT 
      ROUND(
        AVG(LENGTH(ds.key_skills) - LENGTH(replace(ds.key_skills, CHR(9), '')) +1),2
      ) 
    from DSOnly ds
  '''

In [83]:
SqlRequest(text(sql_u6_5))

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [85]:

# Из уже выбранных DS вакансий расчитываем среднюю зарплату
# воспользуемся функцией coalesce
sql_u6_6 = f'''
    WITH DSOnly AS (
    SELECT *
    FROM public.vacancies as v
    WHERE
    {DS_Only}
    )
    
    SELECT
      ROUND(
          AVG(
            coalesce((ds.salary_from+ds.salary_to)/2, ds.salary_from, ds.salary_to) 
            ),0) as average_salary_DS
    FROM DSOnly ds
    WHERE 
     (
      ds.salary_from is not null 
      or ds.salary_to is not null
      )
      and
      ds.experience = 'От 3 до 6 лет'
  '''

In [86]:
SqlRequest(text(sql_u6_6))

,average_salary_ds
0,243115.0


***

#### Вывод по итогам предметного анализа:

1. На портале hh.ru размещена 1771 вакансия имеющая отношение к данным. Это могут быть вакансии для дата-саентистов, аналитиков и т.д.
2. Начинающий дата-саентист (Junior) может рассмотреть всего 51 вакансию. 
3. В требуемых навыках 201 вакансии для дата-саентистов присутствует SQL или postgres. Это 11,3% от общего числа вакансий для DS. Определенно владеть одним или двумя из этих навыков будет полезно для дальнейшего трудоустройства.
4. В требуемых навыках 351 вакансии для дата-саентистов присутствует Python. Это 19,8% от общего числа вакансий для DS. Это самый распостраненный необходимый навык для дальнейшего трудоустройства.
5. В среднем работодатели требуют от соискателя уверенного владения 6,5 навыками, что является большим объемом компетенций. 
6. Средняя предлагаемая зарплата для вакансий по DS составляет 243 115 рублей, что в два раза выше верхней границы средней зарплатной вилки. Высокий уровень зарплат в DS можно объяснить сложностью и продолжительностью подготовки компетентных специалистов, острой нехваткой профессиональных кадров.

# Общий вывод по проекту

Проведем небольшое дополнительное исследование для конкретизации вывода по проекту
1. Определим какие навыки чаще всего встречаются в требованиях к соискателям на должности DS
****

In [200]:
# Для наглядности выведем отдельный столбец с процентным соотношением
# разделим строку с навыками по символу \t
sql_bon_1 = f'''
    WITH DSOnly as (
    SELECT *
    FROM public.vacancies as v
    WHERE
    {DS_Only}
    )
    SELECT
      regexp_split_to_table(ds.key_skills, chr(9)) as skill,
      count(*) as skill_count,
      round(count(*) * 100.0 / count(count(*)) over(),2) as pers
    FROM DSOnly as ds
    GROUP BY skill
    ORDER BY skill_count desc
    LIMIT 15
  '''

In [201]:
SqlRequest(text(sql_bon_1))

,skill,skill_count,pers
0,Python,348,80.93
1,SQL,191,44.42
2,Machine Learning,114,26.51
3,Git,66,15.35
4,Математическая статистика,63,14.65
5,Data Analysis,54,12.56
6,Pandas,52,12.09
7,Data Science,52,12.09
8,ML,49,11.40
9,Английский язык,48,11.16


In [192]:
# Запишем полученный df в переменную
skills = SqlRequest(text(sql_bon_1))

In [193]:
# Строим круговую диаграмму по количеству упоминаний самых распостраненых навыков в вакансиях DS
fig = px.pie(
    data_frame=skills,
    values=skills.skill_count,
    names=skills.skill,
    title='ТОП-15 популярных навыков для DS'
)
fig.update_layout(
legend=dict(
        y=1, 
        x=1,
        title_text=""
    )
)      
fig.show()

2. Определим зависимость средней зарплаты от опыта для двух групп: соискателей на должности DS и всех остальных.
****

In [194]:
# Напишем запрос, который вернет нам среднюю зарплату в зависимости от опыта
# для DS вакансий и Других вакансий
# Создаем условие отбора вакансий не относящихся к DS по аналогии 
Not_DS = '''    
      (
        v.name !~~* any (array[
          '%%data%%scientist%%',
          '%%data%%science%%',
          '%%исследователь%%данных%%',
          '%%machine%%learning%%',
          '%%машинн%%обучен%%'      
        ])
      or (
          v.name ~~* '%%HTML%%'
          and v.name !~~ '%%ML%%'
        )
      )'''
# Используем структуру запроса WITH, для более удобного чтения и понимания.
# Итоговый запрос
sql_bon_2 = f'''
    (
    WITH DSOnly AS (
    SELECT *
    FROM public.vacancies as v
    WHERE
    {Not_DS}
    )
    SELECT
      'Другие вакансии' as vacance_type,
      ds.experience,
      ROUND(
          AVG(
            coalesce((ds.salary_from+ds.salary_to)/2, ds.salary_from, ds.salary_to) 
            ),0) as average_salary_DS
    FROM DSOnly as ds 
    GROUP BY ds.experience  
    ORDER BY average_salary_ds desc NULLS LAST
    )

    UNION

    (
    WITH DSOnly AS (
    SELECT *
    FROM public.vacancies as v
    WHERE
    {DS_Only}
    )
    SELECT
      'вакансии DS' as vacance_type,
      ds.experience,
      ROUND(
          AVG(
            coalesce((ds.salary_from+ds.salary_to)/2, ds.salary_from, ds.salary_to) 
        ),0) as average_salary_DS
    FROM DSOnly as ds 
    GROUP BY ds.experience  
    ORDER BY average_salary_ds desc NULLS LAST
    )
  '''

In [195]:
SqlRequest(text(sql_bon_2))

,vacance_type,experience,average_salary_ds
0,Другие вакансии,От 3 до 6 лет,133211.0
1,вакансии DS,Более 6 лет,NaN
2,вакансии DS,От 1 года до 3 лет,139675.0
3,вакансии DS,От 3 до 6 лет,243115.0
4,Другие вакансии,Более 6 лет,166256.0
5,вакансии DS,Нет опыта,74643.0
6,Другие вакансии,От 1 года до 3 лет,76541.0
7,Другие вакансии,Нет опыта,40472.0


In [196]:
# Запишем полученный df в переменную
df_ds_exp_salary =  SqlRequest(text(sql_bon_2))

In [197]:
# Напишем функцию чтобы добавить сортировку по опыту для графика
def chSort(experience):
    if experience =='Нет опыта':
        return 1
    elif experience == 'От 1 года до 3 лет':
        return 2
    elif experience == 'От 3 до 6 лет':
        return 3
    elif experience == 'Более 6 лет':
        return 4
# Создаем колонку с данными для сортировки
df_ds_exp_salary['Sorting'] = df_ds_exp_salary.apply(lambda x: chSort(x.experience), axis = 1)

# Находим максимальное значение зарплаты, для дополнительного отображения разницы в процентах средней зарплаты от максимума
max = df_ds_exp_salary.average_salary_ds.max() 

# Добавим колонку с данными процентов средней зарплаты от максимума
df_ds_exp_salary['Percent'] = df_ds_exp_salary.average_salary_ds.apply(lambda x: round(x / max, 4))

# Сортируем по новому признаку Sorting
df_ds_exp_salary.sort_values(by = ['Sorting'], inplace=True)

# Строим столбчатую диаграмму
fig = px.bar(
    df_ds_exp_salary,
    x='experience',
    y='average_salary_ds',
    color='vacance_type', 
    hover_data = ['Percent','vacance_type']    
)
fig.update_traces(
    hovertemplate="".join([
        "<b style='color: #393E46'>%{customdata[1]}</b>"    
        "<br>Средняя зарплата (руб): %{y}",
        "<br>",
        "Процент от max ЗП: %{customdata[0]:,.2%}"
    ])
)
fig.update_layout(
    barmode='group',
    hovermode="x unified",
    title='Зависимость средней заработной платы в рублях от опыта соискателя',
    legend=dict(title_text=""),      
    font = dict(size=10), 
    yaxis=dict(
        title_text='Средняя зарплата в рублях', 
        titlefont=dict(size=15)  
    ),
    xaxis=dict(
        title_text='Опыт соискателя', 
        titlefont=dict(size=15)  
    ) 
)
fig.show()


#### Используя дополнительно полученные данные, по итогу выполненного проекта можно сделать следующие выводы:
1. На портале hh.ru для специалистов по DS размещается большое количество вакансий. Это можно объяснить тем, что портал hh.ru является самым популярным ресурсом для поиска работы в РФ, работодатели знают его и активно им пользуются.
2. Самыми распостраненными навыками которыми должен владеть соискатель на должность DS являются  Python, SQL, ML, Математическая Статистика, Git, Data Analysis. Среднее количество навыков, которыми необходимо владеть соискателю равно 6,5. Таким образом соискатель, владеющий всеми шестью навыками скорее всего достаточно быстро найдет работу. Однако общее количество встречающихся навыков в вакансиях DS равно 430! Скорее всего, именно эксклюзивные специалисты могут расчитывать на быстрое трудоустройство и самые высокие зарплаты.
3. Для каждой из трех групп вакансий по опыту работы (нет опыта, от 1 года до 3 лет, от 3 до 6 лет) просматривается одинаковая тенденция. Средняя зарплата для специалистов DS примерно в 2 раза выше чем для всех других специальностей. Также предлагаемая зарплата растет с увеличением опыта соискателя для этих трех групп. Однако для группы более 6 лет вакансии для специалистов DS отсутствуют.
Это можно объяснить тем, что DS является молодой, развивающейся сферой. Стоит отметить, что для всех остальных специальностей в группе более 6 лет тенденция роста зарплаты сохраняется, что наводит на мысль о том, что у дата-саентистов зарплата также растет вместе с опытом.
****